# Задание 6

### Пример входных данных

In [1]:
import pandas as pd
clinton = pd.read_csv("clinton.csv", encoding="utf-8")
trump = pd.read_csv("trump.csv", encoding="utf-8")

In [2]:
trump.head()

,text
0,Join me for a 3pm rally - tomorrow at the Mid-...
1,"Once again, we will have a government of, by a..."
2,"On National #VoterRegistrationDay, make sure y..."
3,Hillary Clinton's Campaign Continues To Make F...
4,"'CNBC, Time magazine online polls say Donald T..."


In [3]:
clinton.head()

,text
0,The question in this election: Who can put the...
1,"Last night, Donald Trump said not paying taxes..."
2,Couldn't be more proud of @HillaryClinton. Her...
3,"If we stand together, there's nothing we can't..."
4,Both candidates were asked about how they'd co...


## Описание задания

Предлагается написать генератор случайных диалогов на основе твитов Трампа (файл <i>"trump.csv"</i>) и Хеллори Клинтон (файл <i>"clinton.csv"</i>).

* Каждый диалог состоит из нескольких "ходов" (<i>turn</i>).
* Каждый такой <i>turn</i> представляет собой цепочку нескольких реплик разных агентов (участников диалога, их может быть больше двух).
* Цепочка реплик представляет собой набор реплик агентов. Первое сообщение в цепочке - <i>"исходное сообщение"</i> на которое отвечают оставшиеся агенты (по одному сообщению за turn).

Программа должна представлять отдельный пакет следующей структуры:
<ul>
<li>run\_generator.py</li>
<li>dialogsgenerator:
<ul>
<li>agent.py</li>
<li>randomdialog.py</li>
<li>\_\_init__.py</li>
</ul></li>
</ul>

<b>URGENT:</b> в работе ЗАПРЕЩАЕТСЯ использовать циклы:
<ul>
<li>Использование 1 for (или while): max 1 балл</li>
<li>Использование до 4 for (или while): max 0.8 баллов</li>
<li>Использование 4+ for (или while): max 0.5 баллов</li>
</ul>

### Файл agent.py

Описание класса агента (один класс для всех агентов, в нашем случае для Трампа и Клинтон).

In [4]:
from collections.abc import Generator
from random import shuffle


class Agent(Generator):

    def __init__(self, kb, name):
        # Инициализация
        self._texts = kb['text']
        self._name = name
        self._create_random_queue()

    def _create_random_queue(self):
        queue = list(range(len(self._texts)))
        shuffle(queue)
        self._iter = iter(queue)

    def __iter__(self):
        return self

    def __next__(self):
        try:
            return f'{self._name}: {self._texts[next(self._iter)]}'
        except StopIteration:
            self._create_random_queue()
            return f'{self._name}: {self._texts[next(self._iter)]}'

    def send(self, msg):
        # Необходимо написать свой собственный метод send для генератора
        return next(self)

    def throw(self, typ=None, val=None, tb=None):
        # Оставляем как есть
        super().throw(typ, val, tb)

    def close(self):
        pass

    def __str__(self):
        # Опишем строковое представление класса
        return f'This generator emulates behaviour of "{self._name}"'

    def __repr__(self):
        return str(self)


#### Пример использования:

In [5]:
from dialogsgenerator import Agent
Agent(clinton, "clinton"), Agent(trump, "trump")

(This generator emulates behaviour of "clinton",
 This generator emulates behaviour of "trump")

### Файл randomdialog.py

Описание класса, генерирующего случайный диалог.

In [6]:
import random
import sys


class RandomDialog(object):

    def __init__(self, agents, max_len=5):
        self._agents = agents
        self._max_len = max_len

    def generate(self):
        """
        Генерирует случайный диалог состоящий из 1-max_len цепочек.
        При генерации вызывает функцию turn.
        Возвращаемый объект является генератором.
        """
        chains_number = random.randrange(self._max_len) + 1
        yield from map(lambda x: list(self.turn()), range(chains_number))

    def turn(self):
        """
        Генерирует одну случайную цепочку следующим образом: выбирается случайный агент.
        Он "говорит" случайное сообщение (msg) из своей Agent.kb (используйте next или send(None)).
        (правила того, как выбирать никак не регулируются, вы можете выбирать случайный твит из Agent.kb никак не учитывая
        переданное msg).
        Это сообщение передается с помощью send (помним, что агент - это объект-генератор).
        Далее получаем ответ от всех агентов и возвращаем (генерируем) их (включая исходное сообщение).
        Возвращаемый объект является генератором.
        """

        speaker_num = random.randrange(len(self._agents))
        speaker = self._agents[speaker_num]
        other_agents = self._agents[:speaker_num] + \
                       self._agents[speaker_num + 1:]
        random.shuffle(other_agents)

        msg = next(speaker)
        yield msg
        yield from map(lambda agent: agent.send(msg), other_agents)

    def eval(self, dialog=None):
        """
        Превращает генератор случайного далога (который возвращается в self.generate())
        в список списков (пример использования далее).
        Возвращаемый объект является списком.
        """
        if dialog is None:
            dialog = self.generate()
        return list(dialog)

    @staticmethod
    def dialog_to_str(dialog):
        return '\n'.join(map(lambda turn: f'turn {turn[0]}:\n' + '\n'.join(
            map(lambda s: '\t' + s, turn[1])), enumerate(dialog)))

    def write(self, dialog=None, target=sys.stdout):
        """
        Записывает результат self.eval() в соответствующий поток.
        """
        if dialog is None:
            dialog = self.eval()
        print(self.dialog_to_str(dialog), file=target)


#### Пример использования:

In [7]:
from dialogsgenerator import RandomDialog
rd = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=2)

In [8]:
generated = rd.generate()
generated

<generator object RandomDialog.generate at 0x7f2af22f6408>

In [9]:
evaled = rd.eval(generated)
evaled

[['trump: Thank you Eau Claire, Wisconsin. \n#VoteTrump on Tuesday, April 5th!\nMAKE AMERICA GREAT AGAIN! https://t.co/JI5JqwHnMC',
  "clinton: .@TomColicchio, thanks for your work on this issue. Your zip code shouldn't determine your ability to access healthy, affordable food."],
 ['clinton: "The death of Alton Sterling is a tragedy, and my prayers are with his family." —Hillary https://t.co/Yky4ZxfbLN',
  'trump: "@SandraR677582: @FLTrumpTeam Florida starts early voting on March 5 extending to March 15. Absentee ballots have been sent out! #Trump2016"']]

In [10]:
rd.write(evaled)

turn 0:
	trump: Thank you Eau Claire, Wisconsin. 
#VoteTrump on Tuesday, April 5th!
MAKE AMERICA GREAT AGAIN! https://t.co/JI5JqwHnMC
	clinton: .@TomColicchio, thanks for your work on this issue. Your zip code shouldn't determine your ability to access healthy, affordable food.
turn 1:
	clinton: "The death of Alton Sterling is a tragedy, and my prayers are with his family." —Hillary https://t.co/Yky4ZxfbLN
	trump: "@SandraR677582: @FLTrumpTeam Florida starts early voting on March 5 extending to March 15. Absentee ballots have been sent out! #Trump2016"


### Файл run_generator.py

Содержит функции для генерации и записи нескольких диалогов. Файл должен быть написан так, чтобы его можно было запускать через командную строку:
<img src="cmd.png">
Описание аргументов представлено на рисунке:
<img src="cmd_help.png">
Для разбора аргументов ипользуйте <a href="https://docs.python.org/3/howto/argparse.html">модуль argparse</a>. Все представленные на рисунке аргументы должны быть обработаны (кроме help, он обрабатывается автоматически модулем argparse).

In [ ]:
import argparse
import sys

import pandas as pd

from dialogsgenerator import Agent
from dialogsgenerator import RandomDialog


def generate(rd, count_dialogs=5):
    """
    Генерирует count_dialogs диалогов с помощью rd.generate().
    Возвращаемый объект является генератором.
    """
    yield from map(lambda i: rd.eval(), range(count_dialogs))


def write(dialogs, target=sys.stdout):
    """
    Записывает сгенерированные диалоги dialogs (это объект-генератор) в поток target.
    """
    print(*map(lambda dialog: f'{f"Dialog {dialog[0]}":_^80}\n' +
                              RandomDialog.dialog_to_str(dialog[1]),
               enumerate(dialogs)), sep='\n', file=target)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('count_dialogs', type=int,
                        help='Amount of dialogs to generate')
    parser.add_argument('max_dialog_len', type=int,
                        help='Maximum amount of turns in each dialog')
    parser.add_argument('-o', '--output', help='output file')
    parser.add_argument('-c', '--clinton_kb', help="Clinton's texts")
    parser.add_argument('-t', '--trump_kb', help="Trump's texts")
    args = parser.parse_args()

    clinton_filename = args.clinton_kb
    if clinton_filename is None:
        clinton_filename = 'clinton.csv'
    trump_filename = args.trump_kb
    if trump_filename is None:
        trump_filename = 'trump.csv'

    if args.count_dialogs <= 0:
        raise ValueError("'count_dialogs' must be positive")
    if args.max_dialog_len <= 0:
        raise ValueError("'max_dialog_len' must be positive")

    clinton = pd.read_csv(clinton_filename)
    trump = pd.read_csv(trump_filename)

    rd = RandomDialog([Agent(clinton, 'clinton'), Agent(trump, 'trump')],
                      args.max_dialog_len)

    if args.output is None:
        fout = sys.stdout
    else:
        fout = open(args.output, 'w')
    with fout:
        write(generate(rd, args.count_dialogs), fout)


#### Пример использования функций из файла:

In [11]:
import sys
from run_generator import generate, write

In [12]:
dialogs = generate(rd, 2)
dialogs

<generator object generate at 0x7f2af22f6480>

In [13]:
write(dialogs, sys.stdout)

____________________________________Dialog 0____________________________________
turn 0:
	clinton: We all know Donald Trump says offensive things about women. Here's what he says about policies that benefit women.
https://t.co/YMV0rvwFnf
	trump: .@Morning_Joe  @mikebarnicle on @realDonaldTrump: He finished 2nd but he made the turn successfully like a pro"
turn 1:
	clinton: “Hillary Clinton has proven herself as a champion of the labor movement." https://t.co/STSP2CRgbp
	trump: The media coverage this morning of the very average Clinton speech and Convention is a joke. @CNN and the little watched @Morning_Joe = SAD!
____________________________________Dialog 1____________________________________
turn 0:
	clinton: 45 years later, I couldn’t be happier to have you by my side. Happy birthday, Bill! -H https://t.co/KDhgLsNr7K
	trump: On immigration, I’m consulting with our immigration officers
&amp; our wage-earners. Hillary Clinton is consulting with Wall Street.
turn 1:
	clinton: These Ca